# ETL

## comienzo importando las librerias necesarias

In [45]:
import pandas as pd 
import numpy as np
from datetime import datetime

## continuo con la carga de los datos

In [46]:
dfa = pd.read_csv('AccidentesAviones.csv')
dfa.sample(5)

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
1283,1283,"October 27, 1951",1900,"Near Flores, Guatemala",Military - Guatemalan Air Force,?,Flores - Guatemala City,Douglas C-47,FAGO961,?,29,25,4,27,23,4,0,Crashed and burned in jungle 170 miles north n...
1885,1885,"February 15, 1961",1005,"Near Brussels, Belgium",Sabena,548,New York City - Brussels,Boeing B-707-320,OO-SJB,17624/92,72,61,11,72,61,11,1,The aircraft was about to touchdown on the run...
1783,1783,"September 08, 1959",?,"Poza Rica, Mexico",Mexicana,?,Mexico City - Merida,Douglas DC-3,?,?,16,13,3,1,1,0,0,An in-flight explosion occurred in the passeng...
4746,4746,"October 17, 2009",1238,"Manila, Philippines",Victoria Air,?,"Manila - Puerto Princesa, Philippines",Douglas DC-3C,RPC550,25737/14292,4,0,4,4,0,4,0,Shortly after take off the crew reported engin...
4545,4545,"March 16, 2005",1357,"Varandey, Russia",Regional Airlines,?,Ufa - Usinsk - Perm - Varandey - Naryan Mar,Antonov 24,RA-46489,27308107,52,45,7,28,26,2,0,"The Russian turboprop carrying oil workers, c..."


## A continuacion presento el **diccionario de datos**

### 
- ***Unnamed: 0:*** Indice para cada registro de accidente de vuelo
- ***fecha:*** Fecha en la que ocurrió el accidente de vuelo
- ***HORA declarada:*** Hora declarada del accidente aéreo
- ***Ruta:*** Ubicación geográfica donde ocurrió el accidente de vuelo
- ***OperadOR:*** Compañía aérea operadora del vuelo
- ***flight_no:*** Número de vuelo
- ***route:*** Ruta del vuelo, indicando los puntos de partida y destino
- ***ac_type:*** Tipo de aeronave involucrada en el accidente
- ***registration:*** Número de registro o matrícula de la aeronave
- ***cn_ln:*** Número de serie de la aeronave
- ***all_aboard:*** Total de personas a bordo (pasajeros y tripulación)
- ***PASAJEROS A BORDO:*** Total de pasajeros a bordo
- ***crew_aboard:*** Total de tripulación a bordo
- ***cantidad de fallecidos:*** Total de personas fallecidas en el accidente (pasajeros y tripulación)
- ***passenger_fatalities:*** Total de pasajeros fallecidos en el accidente
- ***crew_fatalities:*** Total de tripulantes fallecidos en el accidente
- ***ground:*** Total de personas en tierra fallecidas en el accidente
- ***summary:*** Resumen del accidente de vuelo y las circunstancias asociadas

# Observaciones

> Se ha recibido un set de datos que contiene 5008 registros en 18 columnas.
> Ante el primer contacto con los datos se observa:
> * La exitencia de una columna con datos innecesarios:"Unnamed: 0"
> * En algunas columnas aparece el caracter "?" representando la ausencia de dato.

##### Cuantificación de la presencia del caracter "?" en cada una de las columnas y su incidencia

In [47]:
columns = dfa.columns
pd.DataFrame(["Cantidad de '?' en '"+columns+"': "+str(dfa[dfa[columns] == "?"][columns].count()) for columns in columns]).style.hide().hide(axis=1)

Cantidad de '?' en 'Unnamed: 0': 0
Cantidad de '?' en 'fecha': 0
Cantidad de '?' en 'HORA declarada': 1504
Cantidad de '?' en 'Ruta': 5
Cantidad de '?' en 'OperadOR': 10
Cantidad de '?' en 'flight_no': 3682
Cantidad de '?' en 'route': 762
Cantidad de '?' en 'ac_type': 13
Cantidad de '?' en 'registration': 272
Cantidad de '?' en 'cn_ln': 667
Cantidad de '?' en 'all_aboard': 17


## Transformaciones

### Cambio el nombre de cada columna respetando el idioma Ingles

In [50]:
dfa.rename(columns={
    "Unnamed: 0" : "Index",
    "fecha": "Date",
    "HORA declarada": "Event_Time",
    "Ruta": "Location",
    "OperadOR": "Operator",
    "flight_no": "Flight_Number",
    "route": "Route",
    "ac_type": "Aircraft_Type",
    "registration": "Registration",
    "cn_ln": "AC_SerialNumber",
    "all_aboard": "Aboard_Total",
    "PASAJEROS A BORDO": "Passengers_Aboard",
    "crew_aboard": "Crew_Aboard",
    "cantidad de fallecidos": "Total_Fatalities",
    "passenger_fatalities" : "Passenger_Fatalities",
    "crew_fatalities" : "Crew_Fatalities",
    "ground" : "Fatalities_From_Ground",
    "summary" : "Summary"
    }, inplace=True) 

#### Reemplazo de la presencia del caracter "?" en cada una de las columnas por NA Type

In [51]:
dfa.replace('?', pd.NA, inplace=True)

In [52]:
dfa.head()

,Index,Date,Event_Time,Location,Operator,Flight_Number,Route,Aircraft_Type,Registration,AC_SerialNumber,Aboard_Total,Passengers_Aboard,Crew_Aboard,Total_Fatalities,Passenger_Fatalities,Crew_Fatalities,Fatalities_From_Ground,Summary
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,<NA>,Demonstration,Wright Flyer III,<NA>,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,1,"September 07, 1909",<NA>,"Juvisy-sur-Orge, France",<NA>,<NA>,Air show,Wright Byplane,SC1,<NA>,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,<NA>,Test flight,Dirigible,<NA>,<NA>,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,3,"August 06, 1913",<NA>,"Victoria, British Columbia, Canada",Private,<NA>,<NA>,Curtiss seaplane,<NA>,<NA>,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,<NA>,<NA>,Zeppelin L-1 (airship),<NA>,<NA>,20,<NA>,<NA>,14,<NA>,<NA>,0,The airship flew into a thunderstorm and encou...


# En perspectivas de hacer un grafico de Areas y Paises...

In [53]:
# Importo desde el DataSet complementario

df_country = pd.read_csv("Pais.csv")
df_country.rename(columns={'Unnamed: 0' : 'Index'}, inplace=True)

In [54]:
# Combino el DataSet complementario con mi DataFrame final

df_crashes = pd.merge(dfa , df_country, on= 'Index')
df_crashes.head()

,Index,Date,Event_Time,Location,Operator,Flight_Number,Route,Aircraft_Type,Registration,AC_SerialNumber,Aboard_Total,Passengers_Aboard,Crew_Aboard,Total_Fatalities,Passenger_Fatalities,Crew_Fatalities,Fatalities_From_Ground,Summary,country
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,<NA>,Demonstration,Wright Flyer III,<NA>,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly...",United States
1,1,"September 07, 1909",<NA>,"Juvisy-sur-Orge, France",<NA>,<NA>,Air show,Wright Byplane,SC1,<NA>,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...,France
2,2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,<NA>,Test flight,Dirigible,<NA>,<NA>,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...,United States
3,3,"August 06, 1913",<NA>,"Victoria, British Columbia, Canada",Private,<NA>,<NA>,Curtiss seaplane,<NA>,<NA>,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...,Canada
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,<NA>,<NA>,Zeppelin L-1 (airship),<NA>,<NA>,20,<NA>,<NA>,14,<NA>,<NA>,0,The airship flew into a thunderstorm and encou...,United States


In [33]:
# Exportamos a un dataset final de nombre 'AircraftCrash.csv' para el EDA

dfa.to_csv('AircraftCrash.csv', index = False)